## Import


In [ ]:
from stardist.matching import matching
from functions import my_combinations_,filter_combinations_,from_dict_to_folder_path,divide_combi,dict_str,get_image_path,from_dict_to_folder_path,base_dir
import os
import pandas as pd
import tifffile
from tqdm import tqdm

result_path=os.path.join(base_dir+'/results.csv') 

In [ ]:
import ast

def safe_convert(val):
    """Safely convert a value from CSV to its intended type.
    Args:
        val: The value to convert, which may be a string representation of a tuple or a raw value.
    Returns:
        The converted value, or None if the value is NaN.
    """
    if pd.isna(val):
        return None
    if isinstance(val, str) and val.startswith('(') and val.endswith(')'):
        try:
            return ast.literal_eval(val)
        except Exception:
            return val
    return val

done_computations_from_csv=(pd.read_csv(result_path).iloc[:,:-2].map(safe_convert)).to_dict('records')  
combis=my_combinations_()

print('# done computations:',len(done_computations_from_csv))
print('# all computations:',len(combis))
results_to_be_computed=filter_combinations_(new_combinations=combis,done_combinations=done_computations_from_csv)
print('# computations to do:',len(results_to_be_computed))

# computations to do = all combinations - done computations


# done computations: 1996
# all computations: 3718
# computations to do: 1722


## Asses the segemented images

In [ ]:
help(matching)

'''
The documentation for the StarDist library for the matching can be found at:
https://github.com/stardist/stardist/blob/main/stardist/matching.py
'''

Help on function matching in module stardist.matching:

matching(y_true, y_pred, thresh=0.5, criterion='iou', report_matches=False)
    Calculate detection/instance segmentation metrics between ground truth and predicted label images.

    Currently, the following metrics are implemented:

    'fp', 'tp', 'fn', 'precision', 'recall', 'accuracy', 'f1', 'criterion', 'thresh', 'n_true', 'n_pred', 'mean_true_score', 'mean_matched_score', 'panoptic_quality'

    Corresponding objects of y_true and y_pred are counted as true positives (tp), false positives (fp), and false negatives (fn)
    whether their intersection over union (IoU) >= thresh (for criterion='iou', which can be changed)

    * mean_matched_score is the mean IoUs of matched true positives

    * mean_true_score is the mean IoUs of matched true positives but normalized by the total number of GT objects

    * panoptic_quality defined as in Eq. 1 of Kirillov et al. "Panoptic Segmentation", CVPR 2019

    Parameters
    --------

In [ ]:
from joblib import Parallel, delayed
import pandas as pd
# y_true is the annotated ground truth image
y_true = tifffile.imread('/mnt/e/PROJECTS-01/Adrian/Bachelor_project/annotated_data/pos4_t0001_Channel1_annotated_30cells.tif')[77:195,437:1446,654:]
# y_true2 is the cellpose segmented image
y_true2 = tifffile.imread('/mnt/e/PROJECTS-01/Adrian/Bachelor_project/processed_data/Cellpose/no_gauss/t0001_bi1_cellpose_3d_cyto_segmentation_Feyza_cp_masks.tif')

def process_one_combi(combi):
    """Process a single combination and asses the segmentation results.
    Args:
        combi: A dictionary representing the combination to process.
    Returns:
        A dictionary with the results of the processing, including mean matched score and accuracy.
    """
    try:
        divided_combis=[from_dict_to_folder_path(dict_str(combo),path="") for combo in divide_combi(combi)] 
        path_folder=base_dir+"/"+os.path.join(divided_combis[0],divided_combis[1],divided_combis[2])
        condition=lambda file_name:(file_name.endswith('.tiff') or file_name.endswith('.tif'))
        path_file=get_image_path(path_folder,condition)
        if os.path.basename(path_file).startswith("WRONG"):
            mean_matched_score=0
            accuracy=0
        else:
            y_pred=tifffile.imread(path_file)
            mean_matched_score= float((matching(y_true, y_pred)._asdict())["mean_matched_score"])
            accuracy=float((matching(y_true2,y_pred)._asdict())["accuracy"])
        result_entry = combi.copy() 
        result_entry["Mean Matched Score"] = mean_matched_score 
        result_entry["accuracy"]=accuracy
        return result_entry
    except Exception as e:
        print("Error was :")
        print(e)
        print("Combi was :", combi)
        return None

# Process all combinations in parallel
results = Parallel(n_jobs=20)(delayed(process_one_combi)(c) for c in tqdm(results_to_be_computed))
results = [r for r in results if r is not None]

# Save all at once
df = pd.DataFrame(results)
df.to_csv(result_path, mode='a', header=False, index=False)

 29%|██▉       | 500/1722 [11:05<28:40,  1.41s/it]

Error was :
[Errno 2] No such file or directory: '/mnt/e/PROJECTS-01/Adrian/Bachelor_project/GridSearch/Mouse_model/_Threshold_0.4_SigmaSeed_0.1_SigmaWeight_0.5_MinSize_100.0_Alpha_0.8_PixelPitch_None_Mode_multicut/_Beta_0.7_MinSize2_100.0/_'
Combi was : {'Threshold': 0.4, 'SigmaSeed': 0.1, 'SigmaWeight': 0.5, 'MinSize': 100.0, 'Alpha': 0.8, 'PixelPitch': None, 'Mode': 'multicut', 'Beta': 0.7, 'MinSize2': 100.0}


100%|██████████| 1722/1722 [39:05<00:00,  1.36s/it]


## Visualisation 

In [ ]:
import pandas as pd

# Sort and print the top 5 combinations by 'Mean Matched Score' descending
df = pd.read_csv(result_path)
df_sorted = df.sort_values(by='Mean Matched Score', ascending=False)
top_5 = df_sorted.head(5)
print(top_5)

# Sort and print the top 5 combinations by 'accuracy' descending
df_sorted_2 = df.sort_values(by='accuracy', ascending=False)
top_5_2 = df_sorted_2.head(5)
print(top_5_2)

     Threshold  SigmaSeed  SigmaWeight  MinSize  Alpha     PixelPitch  Mode  \
755        0.4        0.2          0.0     90.0    0.7  (1.441, 1, 1)  gasp   
754        0.4        0.2          0.0     90.0    0.7  (1.441, 1, 1)  gasp   
745        0.4        0.2          0.0     90.0    0.7  (1.441, 1, 1)  gasp   
744        0.4        0.2          0.0     90.0    0.7  (1.441, 1, 1)  gasp   
656        0.4        0.2          0.0    100.0    0.7  (1.441, 1, 1)  gasp   

     Beta  MinSize2  Treshold2  Instances  Mean Matched Score  accuracy  
755   0.7      75.0        0.2       True             0.78103  0.070785  
754   0.7      75.0        0.2      False             0.78103  0.070798  
745   0.7     100.0        0.1       True             0.78103  0.074730  
744   0.7     100.0        0.1      False             0.78103  0.074743  
656   0.7      75.0        0.1      False             0.78103  0.074579  
      Threshold  SigmaSeed  SigmaWeight  MinSize  Alpha     PixelPitch  \
1991   

### Figures 
Created with https://claude.ai/new

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

class ParameterAnalyzer:
    def __init__(self, csv_path):
        """Initialize the analyzer with data from CSV file"""
        self.df = pd.read_csv(csv_path)
        self.prepare_data()
        
    def prepare_data(self):
        """Clean and prepare the data for analysis"""
        # Convert boolean columns
        bool_cols = ['Instances']
        for col in bool_cols:
            if col in self.df.columns:
                self.df[col] = self.df[col].map({'TRUE': True, 'FALSE': False})
        
        # Identify parameter columns (exclude metrics)
        self.metric_cols = ['Mean Matched Score', 'accuracy']
        self.param_cols = [col for col in self.df.columns 
                          if col not in self.metric_cols and col != 'PixelPitch']
        
        # Create filtered dataset (non-zero values)
        self.df_filtered = self.df[
            (self.df['Mean Matched Score'] > 0) | (self.df['accuracy'] > 0)
        ].copy()
        
        print(f"Total combinations: {len(self.df)}")
        print(f"Non-zero combinations: {len(self.df_filtered)}")
        print(f"Success rate: {len(self.df_filtered)/len(self.df)*100:.1f}%")
    
    def plot_correlation_heatmap(self, use_filtered=True):
        """Create correlation heatmap of all numeric parameters"""
        data = self.df_filtered if use_filtered else self.df
        
        # Select only numeric columns for correlation
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        corr_matrix = data[numeric_cols].corr()
        
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', 
                   center=0, square=True, linewidths=0.5, cbar_kws={"shrink": .8})
        
        plt.title('Parameter Correlation Matrix', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    def plot_parameter_heatmap(self, x_param, y_param, metric='Mean Matched Score', 
                              use_filtered=True, aggregation='mean'):
        """Create a heatmap for two parameters colored by a metric"""
        data = self.df_filtered if use_filtered else self.df
        
        if len(data) == 0:
            print("No data to plot after filtering")
            return
        
        # Create pivot table
        if aggregation == 'mean':
            pivot_data = data.pivot_table(values=metric, index=y_param, 
                                        columns=x_param, aggfunc='mean')
        elif aggregation == 'max':
            pivot_data = data.pivot_table(values=metric, index=y_param, 
                                        columns=x_param, aggfunc='max')
        else:  # count
            pivot_data = data.pivot_table(values=metric, index=y_param, 
                                        columns=x_param, aggfunc='count')
        
        plt.figure(figsize=(12, 8))
        
        # Create heatmap
        sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='RdYlGn', 
                   cbar_kws={'label': metric}, linewidths=0.5)
        
        plt.title(f'{metric} by {x_param} vs {y_param} '
                 f'({aggregation} aggregation)', fontsize=14, fontweight='bold')
        plt.xlabel(x_param, fontsize=12)
        plt.ylabel(y_param, fontsize=12)
        plt.tight_layout()
        plt.show()
    
    def plot_parameter_distribution(self, use_filtered=True):
        """Plot distribution of all parameters"""
        data = self.df_filtered if use_filtered else self.df
        
        fig, axes = plt.subplots(3, 3, figsize=(15, 12))
        axes = axes.ravel()
        
        for i, param in enumerate(self.param_cols[:9]):  # Plot first 9 parameters
            if i < len(axes):
                if data[param].dtype in ['object', 'bool']:
                    # For categorical data
                    value_counts = data[param].value_counts()
                    axes[i].bar(range(len(value_counts)), value_counts.values)
                    axes[i].set_xticks(range(len(value_counts)))
                    axes[i].set_xticklabels(value_counts.index, rotation=45)
                else:
                    # For numeric data
                    axes[i].hist(data[param], bins=20, alpha=0.7, edgecolor='black')
                
                axes[i].set_title(param, fontweight='bold')
                axes[i].grid(True, alpha=0.3)
        
        # Hide empty subplots
        for i in range(len(self.param_cols), len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle('Parameter Distributions', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    def plot_performance_by_parameter(self, param, metric='Mean Matched Score', use_filtered=True):
        """Plot performance metric by a specific parameter"""
        data = self.df_filtered if use_filtered else self.df
        
        plt.figure(figsize=(12, 6))
        
        if data[param].dtype in ['object', 'bool'] or data[param].nunique() < 10:
            # Box plot for categorical or few unique values
            sns.boxplot(data=data, x=param, y=metric)
            plt.xticks(rotation=45)
        else:
            # Scatter plot for continuous variables
            plt.scatter(data[param], data[metric], alpha=0.6)
            # Add trend line
            z = np.polyfit(data[param], data[metric], 1)
            p = np.poly1d(z)
            plt.plot(data[param], p(data[param]), "r--", alpha=0.8)
        
        plt.title(f'{metric} by {param}', fontsize=14, fontweight='bold')
        plt.xlabel(param, fontsize=12)
        plt.ylabel(metric, fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def plot_top_combinations(self, metric='Mean Matched Score', top_n=10):
        """Plot the top N combinations"""
        top_combos = self.df_filtered.nlargest(top_n, metric)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Bar plot of top combinations
        combo_labels = [f"Combo {i+1}" for i in range(len(top_combos))]
        ax1.bar(combo_labels, top_combos[metric], color='skyblue', edgecolor='navy')
        ax1.set_title(f'Top {top_n} Combinations by {metric}', fontweight='bold')
        ax1.set_ylabel(metric)
        ax1.tick_params(axis='x', rotation=45)
        ax1.grid(True, alpha=0.3)
        
        # Heatmap of parameter values for top combinations
        param_subset = ['Threshold', 'Beta', 'Alpha', 'MinSize', 'MinSize2']  # Key parameters
        available_params = [p for p in param_subset if p in top_combos.columns]
        
        if available_params:
            heatmap_data = top_combos[available_params].T
            sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='viridis', 
                       ax=ax2, cbar_kws={'label': 'Parameter Value'})
            ax2.set_title('Parameter Values for Top Combinations', fontweight='bold')
            ax2.set_xlabel('Combination Rank')
        
        plt.tight_layout()
        plt.show()
        
        return top_combos
    
    def plot_metric_comparison(self):
        """Compare the two metrics against each other"""
        if len(self.df_filtered) == 0:
            print("No filtered data available for comparison")
            return
        
        plt.figure(figsize=(10, 8))
        
        # Scatter plot
        plt.scatter(self.df_filtered['Mean Matched Score'], 
                   self.df_filtered['accuracy'], alpha=0.6, s=50)
        
        # Add correlation coefficient
        corr = self.df_filtered['Mean Matched Score'].corr(self.df_filtered['accuracy'])
        plt.title(f'Mean Matched Score vs Accuracy\n(Correlation: {corr:.3f})', 
                 fontsize=14, fontweight='bold')
        
        plt.xlabel('Mean Matched Score', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # Add trend line
        if len(self.df_filtered) > 1:
            z = np.polyfit(self.df_filtered['Mean Matched Score'], 
                          self.df_filtered['accuracy'], 1)
            p = np.poly1d(z)
            x_trend = np.linspace(self.df_filtered['Mean Matched Score'].min(),
                                 self.df_filtered['Mean Matched Score'].max(), 100)
            plt.plot(x_trend, p(x_trend), "r--", alpha=0.8, linewidth=2)
        
        plt.tight_layout()
        plt.show()
    
    def comprehensive_analysis(self):
        """Run a comprehensive analysis with multiple visualizations"""
        print("=== COMPREHENSIVE PARAMETER ANALYSIS ===\n")
        
        # 1. Basic statistics
        print("1. BASIC STATISTICS:")
        print(f"Best Mean Matched Score: {self.df['Mean Matched Score'].max():.6f}")
        print(f"Best Accuracy: {self.df['accuracy'].max():.6f}")
        print(f"Average Mean Matched Score (non-zero): {self.df_filtered['Mean Matched Score'].mean():.6f}")
        print(f"Average Accuracy (non-zero): {self.df_filtered['accuracy'].mean():.6f}")
        print("\n" + "="*50 + "\n")
        
        # 2. Correlation heatmap
        print("2. PLOTTING: Correlation Matrix")
        self.plot_correlation_heatmap()
        
        # 3. Parameter distributions
        print("3. PLOTTING: Parameter Distributions")
        self.plot_parameter_distribution()
        
        # 4. Key parameter heatmaps
        print("4. PLOTTING: Key Parameter Heatmaps")
        key_combinations = [
            ('Threshold', 'Beta'),
            ('Alpha', 'Beta'),
            ('MinSize', 'MinSize2'),
            ('Threshold', 'Alpha')
        ]
        
        for x_param, y_param in key_combinations:
            if x_param in self.param_cols and y_param in self.param_cols:
                print(f"   - {x_param} vs {y_param}")
                self.plot_parameter_heatmap(x_param, y_param, 'Mean Matched Score')
        
        # 5. Performance by key parameters
        print("5. PLOTTING: Performance by Key Parameters")
        key_params = ['Threshold', 'Beta', 'Alpha', 'MinSize']
        for param in key_params:
            if param in self.param_cols:
                print(f"   - Performance by {param}")
                self.plot_performance_by_parameter(param, 'Mean Matched Score')
        
        # 6. Top combinations
        print("6. PLOTTING: Top Combinations")
        top_combos = self.plot_top_combinations('Mean Matched Score', 15)
        
        # 7. Metric comparison
        print("7. PLOTTING: Metric Comparison")
        self.plot_metric_comparison()
        
        # 8. Best combination details
        print("8. BEST COMBINATION DETAILS:")
        best_combo = self.df.loc[self.df['Mean Matched Score'].idxmax()]
        print("\nBest Mean Matched Score combination:")
        for param in self.param_cols:
            if param in best_combo:
                print(f"  {param}: {best_combo[param]}")
        print(f"  Mean Matched Score: {best_combo['Mean Matched Score']:.6f}")
        print(f"  Accuracy: {best_combo['accuracy']:.6f}")

In [ ]:

# initialize the analyzer with the path to the CSV file
analyzer = ParameterAnalyzer("results.csv")

#Choose what you want to do


analyzer.comprehensive_analysis()

# Individual plots
analyzer.plot_correlation_heatmap()
analyzer.plot_parameter_heatmap('Alpha', 'Beta', 'Mean Matched Score')
analyzer.plot_parameter_heatmap('Alpha', 'Beta', 'accuracy')
analyzer.plot_performance_by_parameter('Alpha', 'Mean Matched Score')
analyzer.plot_performance_by_parameter('Beta', 'Mean Matched Score')
analyzer.plot_top_combinations('Mean Matched Score', 15)
analyzer.plot_top_combinations('accuracy', 15)